<a href="https://colab.research.google.com/github/malkaby/dom-testing-library/blob/main/ML_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import math as m
from IPython.display import Markdown as md
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
import json
import re
import string
import pandas as pd
from sklearn.cluster import KMeans, MiniBatchKMeans
import gensim
from gensim.parsing.preprocessing import preprocess_documents
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
from nltk import word_tokenize
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from nltk.stem import WordNetLemmatizer
print('Hello, world!')

Hello, world!


In [ ]:
class ProcessData:
    """
    Load, remove duplicates and copy just important columns to new dataset.
    """
    def __init__(self, file : str, base_col : str, types: dict=None, to_copy : list=None, dup_drop : list=None, conditions : list=None) -> None:
        self.file = file;
        self.dup_drop = dup_drop;
        self.to_copy = to_copy;
        self.conditions = conditions;
        self.base_col = base_col;
        
    def load_dataset(self) -> object:
        """ 
        Load dataset from excel file.
        """
        return pd.read_csv(self.file)
    
    def copy_columns(self) -> object:
        """ 
        Copy just columns that are important.
        """
        data = self.drop_duplicates()
        return data[self.to_copy].copy()
  
    def drop_duplicates(self) -> object:
        """ 
        Drop duplicates in columns.
        """
        if self.conditions == None:
            data = self.load_dataset()
        else:
            data = self.drop_row_condition()
        for i in range(len(self.dup_drop)):
            data = data.drop_duplicates(subset=[self.dup_drop[i]], keep="last")
        return data.dropna();
    
    def drop_row_condition(self) -> object:
        """
        Drop row based on condition 
        """
        data = self.load_dataset()
        for i in range(len(self.conditions)):
            data = data[(data[self.base_col] != self.conditions[i])]
        return data;

In [ ]:
dup_drop = ['Ticket','Problem Abstract']; # List of Columns from which we want to drop duplicates
to_copy = ['Ticket','Problem Abstract']; # The final dataset will contain just these columns

file = '/content/ML_DATASET_Hackathon_cleaned.csv'; # path/to/file

## Call the ProcessData call
data = ProcessData(file, base_col = None, to_copy = to_copy, dup_drop = dup_drop, conditions = None).copy_columns()

## Save the data as pickle file
data.to_pickle('/content/proc.pkl')

In [ ]:
data.head()

,Ticket,Problem Abstract
7,306441232,GTAC login failure\n\n
8,306440903,not able to login IBM LAN device using GTAC\n\n
9,306446422,Don't have the access to GTAC TACACS Log Viewe...
10,306447433,GTAC domain password rules
18,306458799,"mechid m32239 is locked, suspect Cisco DNAC......"


In [ ]:
class Tfid:
    def __init__(self, col_name : str, data : object):
        """ https://radimrehurek.com/gensim/models/tfidfmodel.html. """
        self.col_name = col_name
        self.data = data
    
    def create_dictonary(self):
        processed_corpus = preprocess_documents(self.data[self.col_name].values)
        dictionary = gensim.corpora.Dictionary(processed_corpus)
        dict_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
        return dict_corpus, dictionary

## Load the pickle file    
data = pd.read_pickle('/content/proc.pkl')

## Get the dict_corpus and the dictionary for the "Problem Abstract"
proc_corpus, dictionary  = Tfid('Problem Abstract',data).create_dictonary()
data.head()

print(proc_corpus)
print (dictionary)

## Run the TfidfModel on the proc_corpus
tfidf = TfidfModel(proc_corpus, slope = 0.3)

# Get the indexes of ...
index = gensim.similarities.MatrixSimilarity(tfidf[proc_corpus])

print (index)

[[(0, 1), (1, 1), (2, 1)], [(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)], [(1, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)], [(1, 1), (9, 1), (13, 1), (14, 1)], [(4, 1), (10, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1)], [(23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1)], [(1, 1), (33, 1), (34, 1), (35, 1)], [(9, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1)], [(24, 1), (25, 1), (28, 1), (30, 1), (42, 1), (43, 1), (44, 1), (45, 1)], [(46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 2), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 2), (61, 1), (62, 1), (63, 1), (64, 2), (65, 1)], [(1, 1), (13, 2), (66, 1), (67, 1), (68, 1), (69, 1), (70, 2)], [(71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1)], [(3, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1)], [(1, 1), (19, 1), (84, 1)], [(85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1)], 

In [ ]:
## This to be provided be the user
new_problem_abstract = 'My router is down'

## Using preprocessing from the gensim 
new_problem_abstract_processed = gensim.parsing.preprocessing.preprocess_string(new_problem_abstract)

## Converting the new_problem_abstract to doc2bow
new_vec_problem_abstract = dictionary.doc2bow(new_problem_abstract_processed)
print (new_vec_problem_abstract)
vec_bow_tfidf = tfidf[new_vec_problem_abstract]
print (vec_bow_tfidf)
similiarities = index[vec_bow_tfidf]
print (similiarities)

for sim in sorted(enumerate(similiarities), key=lambda item: -item[1])[:10]:
    print(f"\"{data['Problem Abstract'].iloc[sim[0]]}\" : is similar to \"{new_problem_abstract}\" by: {str(sim[1])}")

[(345, 1)]
[(345, 1.0)]
[0. 0. 0. ... 0. 0. 0.]
"Unable to access routers

" : is similar to "My router is down" by: 0.7466861
"Unable to access router

" : is similar to "My router is down" by: 0.7466861
"Not able to login to routers

" : is similar to "My router is down" by: 0.6061293
"Unable to access AVPN router

" : is similar to "My router is down" by: 0.53662384
"No auto ticketing when routers went down" : is similar to "My router is down" by: 0.51829064
"GTAC account not working in Switches and Routers

" : is similar to "My router is down" by: 0.5174085
"TACACS RO access to MRS routers" : is similar to "My router is down" by: 0.5055248
"Router down without autodetect" : is similar to "My router is down" by: 0.47612485
"User unable to access certain routers.

" : is similar to "My router is down" by: 0.44519266
"Hi team. I have issue on logging to some of AVPN managed routers like USSNCPHARSC01R and USSADGCHWCA01R. It keeps me asking for GTAC PIN Passcode. And even I re-try mul